In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [4]:
temp_data_train = pd.read_csv('../Data/train_data.csv')
temp_data_test = pd.read_csv('../Data/test_data.csv')
# Convert the 'date' column to datetime
temp_data_train['date'] = pd.to_datetime(temp_data_train['date'])
temp_data_test['date'] = pd.to_datetime(temp_data_test['date'])

T_actual_train = temp_data_train['mean'].to_numpy()
T_actual_test = temp_data_test['mean'].to_numpy()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# 1. Load your parameters (shape assumed to be (N,))
S = np.load("S_seasonal.npy")         # Seasonal component
k = np.load("k_timevarying.npy")        # Time-varying parameter
sigma = np.load("sigma_timevarying.npy")  # Time-varying volatility

# 2. Forecast settings
forecast_end = 365  # number of forecast time steps
forecast_start = 0

# 3. Number of Monte Carlo simulation paths
n_paths = 1000  # you can change this number as needed

# 4. Generate the Monte Carlo paths
T_all = np.zeros((n_paths, forecast_end))
initial_value = T_actual_train[-1]  # starting with the last observed value from training data
T_all[:, 0] = initial_value

# Simulate the paths
for t in range(1, forecast_end):
    eps = np.random.randn(n_paths)  # random shock for each simulation path at time t
    T_all[:, t] = S[t] + (1 + k[t-1]) * (T_all[:, t-1] - S[t-1]) + sigma[t] * eps

# Average forecast over all paths at each time step
T_avg = np.mean(T_all, axis=0)

# -- Compute error statistics between T_avg and T_actual_test --
mae = mean_absolute_error(T_actual_test, T_avg)
mse = mean_squared_error(T_actual_test, T_avg)
rmse = np.sqrt(mse)
corr_coef = np.corrcoef(T_actual_test, T_avg)[0, 1]

print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("Correlation Coefficient:", corr_coef)

# -- Plot the Actual Test Data vs. MC Average Forecast --
plt.figure(figsize=(10, 6))
plt.plot(T_actual_test, label="Actual Test Data", linewidth=2, color='blue')
plt.plot(T_avg, label="MC Average Forecast", linewidth=2, linestyle="--", color='orange')
plt.xlabel("Time Step (Days)")
plt.ylabel("Value")
plt.title("Comparison: Actual Test Data vs. MC Average Forecast")
plt.legend()
plt.show()
